In [1]:
mutable struct Node
    data
    prev::Union{Node, Nothing}
    next::Union{Node, Nothing}
end

Node(0, Node(#= circular reference @-1 =#), Node(#= circular reference @-1 =#))

In [3]:
L = circlist(0)
@show L
@show L.prev
@show L.next

L = Node(0, Node(#= circular reference @-1 =#), Node(#= circular reference @-1 =#))
L.prev = Node(0, Node(#= circular reference @-1 =#), Node(#= circular reference @-1 =#))
L.next = Node(0, Node(#= circular reference @-1 =#), Node(#= circular reference @-1 =#))


Node(0, Node(#= circular reference @-1 =#), Node(#= circular reference @-1 =#))

In [42]:
function circlist(data)
    n = Node(data, nothing, nothing)
    n.prev = n
    n.next = n
    return n
end

function display(cl::Node)
    start = cl
    while true
        print(cl.data, " ")
        cl = cl.next
        cl == start && break
    end
    println()
end

function display_backward(cl::Node)
    start = cl
    while true
        print(cl.data, " ")
        cl = cl.prev
        cl == start && break
    end
    println()
end

function length(cl::Node)
    n = 0
    s = cl
    while true
        cl = cl.next
        n += 1
        s == cl && break
    end
    return n
end

# insert a new node after the current node, returns new node
function insert!(cl::Node, data) 
    n = Node(data, cl, cl.next)
    cl.next = n
    n.next.prev = n
    return n
end

# delete current node, returns previous node
function remove!(cl::Node)
    length(cl) == 1 && error("cannot remove last item in circular list")
    cl.prev.next = cl.next
    cl.next.prev = cl.prev
    return cl.prev
end

# navigate forward/backward in the circular list
function shift(cl::Node, steps::Int, direction::Symbol)
    for i in 1:steps
        if direction == :forward 
            cl = cl.next
        elseif direction == :backward
            cl = cl.prev
        else
            error("Wrong direction: $direction")
        end
    end
    return cl
end

# unit testing
Q = L = circlist(0)
display(L)

L = insert!(L, 1)
display(Q)

L = insert!(L, 2)
display(Q)
display_backward(Q)

shift(Q, 2, :forward) |> display

length(Q) |> println

shift(Q, 1, :forward) |> remove! |> display

W = circlist(0)
for i in 1:10 insert!(W, i); W = W.next; end
display(W)
display(shift(W, 5, :forward))
display(shift(W, 3, :backward))

0 
0 1 
0 1 2 
0 2 1 
2 0 1 
3
0 2 
10 0 1 2 3 4 5 6 7 8 9 
4 5 6 7 8 9 10 0 1 2 3 
7 8 9 10 0 1 2 3 4 5 6 


In [58]:
using Dates

function play(num_players, last_marble, chunk)
    leaderboard = fill(0, num_players)
    game = curr_marble = circlist(0)
    marble = 0
    while marble < last_marble
        marble += 1
        marble % chunk == 0 && (println(Dates.now(), ": marble $marble processed"), flush(stdout))
        if marble % 23 == 0
            player = (marble % num_players) + 1
            leaderboard[player] += marble   # win current marble
            curr_marble = shift(curr_marble, 7, :backward)
#             print("after shifted backward => ") ; display(curr_marble)
            leaderboard[player] += curr_marble.data  # win the other marble
            curr_marble = remove!(curr_marble)
            curr_marble = shift(curr_marble, 1, :forward)
        else
            curr_marble = shift(curr_marble, 1, :forward)
            curr_marble = insert!(curr_marble, marble)
        end
#         print("after placing $marble: curr_marble = "); display(curr_marble)
    end
#     display(game)
    return (scores = leaderboard, 
        highest = maximum(leaderboard),
        num_players = num_players,
        last_marble = last_marble
        )
end
# [4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
# [5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
# [6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
# [7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
# [7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
# @show play(10, 1618)
# @show play(13, 7999)
# @show play(17, 1104)
@time play(431, 7095000, 100_000)

2018-12-08T22:46:38.812: marble 100000 processed
2018-12-08T22:46:41.568: marble 200000 processed
2018-12-08T22:46:46.483: marble 300000 processed
2018-12-08T22:46:53.198: marble 400000 processed
2018-12-08T22:47:01.583: marble 500000 processed
2018-12-08T22:47:12.522: marble 600000 processed
2018-12-08T22:47:24.868: marble 700000 processed
2018-12-08T22:47:38.831: marble 800000 processed
2018-12-08T22:47:54.633: marble 900000 processed
2018-12-08T22:48:12.333: marble 1000000 processed
2018-12-08T22:48:32.209: marble 1100000 processed
2018-12-08T22:48:53.824: marble 1200000 processed
2018-12-08T22:49:17.812: marble 1300000 processed
2018-12-08T22:49:43.276: marble 1400000 processed
2018-12-08T22:50:17.392: marble 1500000 processed
2018-12-08T22:51:03.027: marble 1600000 processed
2018-12-08T22:51:39.602: marble 1700000 processed
2018-12-08T22:52:13.4: marble 1800000 processed
2018-12-08T22:52:53.411: marble 1900000 processed
2018-12-08T22:53:35.982: marble 2000000 processed
2018-12-08T

(scores = [3329318022, 3338084362, 3341579397, 3336818909, 3347592587, 3340225602, 3338953147, 3343797117, 3340419992, 3340629970  …  3325297181, 3324569457, 3319641009, 3310406271, 3312373465, 3320100167, 3324828503, 3324005306, 3331266930, 3321826335], highest = 3350093681, num_players = 431, last_marble = 7095000)